In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pickle

In [2]:
# filename = "../input/radiant-images-features-extraction/dict_predictions.pickle"
filename = "../input/radiant-neighbours-features-extraction/dict_predictions.pickle"

with open(filename, 'rb') as handle:
    dict_predictions = pickle.load(handle)
dict_predictions.keys()

dict_keys(['filenames', 'predictions'])

In [3]:
df = pd.DataFrame()
df["filename"] = dict_predictions["filenames"]
predictions = dict_predictions["predictions"]
predictions.shape # 7426 x 5

(37130, 10)

In [4]:
class_indices = {'AnnualCrop': 0,
 'Forest': 1,
 'HerbaceousVegetation': 2,
 'Highway': 3,
 'Industrial': 4,
 'Pasture': 5,
 'PermanentCrop': 6,
 'Residential': 7,
 'River': 8,
 'SeaLake': 9}

df["dataset"] = df["filename"].apply(lambda f: f.split("/")[0])
df["tile_id"] = df["filename"].apply(lambda f: f.split("/")[-1].split("_")[0])
df["field_id"] = df["filename"].apply(lambda f: f.split("/")[-1].split("_")[1])
df["field_neigbour"] = df["filename"].apply(lambda f: f.split("/")[-1].split("_")[-1].split(".")[0])
for c, i in class_indices.items():
    col = f"prob_{c}"
    df[col] = predictions[:, i]
    
df.tail()

,filename,dataset,tile_id,field_id,field_neigbour,prob_AnnualCrop,prob_Forest,prob_HerbaceousVegetation,prob_Highway,prob_Industrial,prob_Pasture,prob_PermanentCrop,prob_Residential,prob_River,prob_SeaLake
37125,train/ffe8c_4617_east.png,train,ffe8c,4617,east,2.532709e-06,2.661425e-07,0.997593,4.809851e-07,3.114003e-07,0.000039,0.002364,4.357606e-07,7.411898e-08,6.026280e-11
37126,train/ffe8c_4617_main.png,train,ffe8c,4617,main,4.763058e-07,6.263498e-08,0.999058,6.762929e-08,5.108151e-08,0.000010,0.000931,7.588582e-08,9.494311e-09,3.753684e-12
37127,train/ffe8c_4617_north.png,train,ffe8c,4617,north,3.819961e-05,1.328420e-04,0.994182,9.572344e-06,5.019935e-06,0.001247,0.004360,1.883443e-05,6.543347e-06,6.330155e-08
37128,train/ffe8c_4617_south.png,train,ffe8c,4617,south,2.168239e-06,1.215094e-06,0.998106,4.114931e-07,2.875667e-07,0.000056,0.001833,5.609279e-07,1.046398e-07,1.060724e-10
37129,train/ffe8c_4617_west.png,train,ffe8c,4617,west,6.521116e-07,2.047490e-07,0.999310,4.647695e-08,3.225862e-08,0.000015,0.000674,7.392907e-08,1.078833e-08,8.853163e-12


In [5]:
df.describe()

,prob_AnnualCrop,prob_Forest,prob_HerbaceousVegetation,prob_Highway,prob_Industrial,prob_Pasture,prob_PermanentCrop,prob_Residential,prob_River,prob_SeaLake
count,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04,3.713000e+04
mean,4.279323e-02,1.936422e-01,5.968204e-01,3.280117e-03,4.764452e-03,9.345557e-02,3.708471e-02,1.229032e-02,2.513157e-03,1.335588e-02
std,1.430263e-01,3.555977e-01,4.472903e-01,4.087940e-02,5.025233e-02,2.192536e-01,1.108005e-01,8.613276e-02,1.806873e-02,4.972276e-02
min,6.266000e-19,2.034271e-35,2.191362e-20,8.520354e-24,7.065926e-28,1.513025e-24,6.607406e-28,3.015967e-19,7.571133e-18,0.000000e+00
25%,9.093932e-06,4.744857e-07,2.818206e-02,6.457985e-07,2.993462e-07,8.571205e-05,9.105910e-04,5.636025e-07,2.718676e-07,3.817054e-10
50%,1.967956e-04,5.212470e-05,9.197502e-01,1.663786e-05,6.026996e-06,1.302490e-03,4.470863e-03,1.011224e-05,1.480930e-05,1.911886e-07
75%,7.191784e-03,1.337638e-01,9.941939e-01,3.493063e-04,6.308433e-05,4.305145e-02,1.597355e-02,1.302904e-04,1.268577e-03,2.084314e-03
max,9.993271e-01,9.999999e-01,9.999387e-01,9.997310e-01,9.989857e-01,9.999998e-01,9.997436e-01,9.999548e-01,9.999098e-01,8.170304e-01


In [6]:
prob_cols = [c for c in df.columns if "prob" in c]
df["alias_name"] = df["filename"].apply(lambda f: "_".join(f.split("_")[:-1]))

df_pivot = df.pivot(index='alias_name', columns="field_neigbour", values=prob_cols).reset_index()
df_pivot["dataset"] = df_pivot["alias_name"].apply(lambda f: f.split("/")[0])
df_pivot["tile_id"] = df_pivot["alias_name"].apply(lambda f: f.split("/")[-1].split("_")[0])
df_pivot["field_id"] = df_pivot["alias_name"].apply(lambda f: f.split("/")[-1].split("_")[1])

df_pivot

alias_name prob_AnnualCrop                          \
field_neigbour                              east          main     north   
0                test/001c1_6533        0.002468  3.285539e-01  0.328554   
1                test/004fa_3989        0.004605  5.620613e-04  0.000014   
2                test/005fe_2018        0.116594  6.338482e-02  0.019930   
3                test/00720_6643        0.146753  4.772997e-02  0.016439   
4                test/00720_6646        0.029916  1.001395e-01  0.005639   
...                          ...             ...           ...       ...   
7421            train/ffe8c_3850        0.000002  4.063216e-07  0.000014   
7422            train/ffe8c_3851        0.000007  1.198573e-06  0.000035   
7423            train/ffe8c_4045        0.000003  5.653654e-03  0.005654   
7424            train/ffe8c_4046        0.000002  9.760111e-03  0.009760   
7425            train/ffe8c_4617        0.000003  4.763058e-07  0.000038   

                                             prob_Forest                \
field_neigbour         south          west          east          main   
0               2.386288e-02  2.576364e-06  9.640521e-01  7.572761e-02   
1               2.318358e-07  4.931064e-04  1.848391e-01  5.169205e-05   
2               2.116136e-02  1.158904e-01  2.461883e-02  2.324435e-01   
3               6.728244e-02  4.772997e-02  3.167723e-01  6.463121e-01   
4               4.707675e-02  3.091487e-01  5.585895e-01  4.680143e-01   
...                      ...           ...           ...           ...   
7421            6.664902e-07  1.450067e-06  1.201619e-06  9.034929e-08   
7422            9.799763e-07  2.303124e-06  4.589290e-06  4.405366e-07   
7423            7.098048e-04  1.459399e-02  3.824441e-06  9.431757e-01   
7424            7.049917e-04  1.459399e-02  1.684941e-06  8.935382e-01   
7425            2.168239e-06  6.521116e-07  2.661425e-07  6.263498e-08   

                                            ...    prob_River                \
field_neigbour         north         south  ...         south          west   
0               7.572761e-02  8.021359e-01  ...  2.035267e-03  9.528738e-09   
1               6.297568e-07  2.511992e-08  ...  3.371843e-09  9.879674e-05   
2               7.923018e-01  7.707615e-01  ...  2.349643e-03  1.418961e-02   
3               8.660864e-01  2.550774e-01  ...  8.487009e-03  4.290956e-03   
4               9.252840e-01  5.137508e-01  ...  6.597087e-03  1.660051e-02   
...                      ...           ...  ...           ...           ...   
7421            3.767270e-05  4.072448e-07  ...  2.905978e-08  3.668340e-08   
7422            3.522110e-05  8.732323e-07  ...  5.147903e-08  5.973239e-08   
7423            9.431757e-01  9.907694e-01  ...  2.994075e-05  2.479941e-03   
7424            8.935382e-01  9.893566e-01  ...  3.729520e-05  2.479941e-03   
7425            1.328420e-04  1.215094e-06  ...  1.046398e-07  1.078833e-08   

                prob_SeaLake                                            \
field_neigbour          east          main         north         south   
0               1.200161e-02  8.295685e-02  8.295685e-02  9.351956e-02   
1               9.389773e-04  4.503298e-07  6.654428e-10  8.192675e-13   
2               3.744161e-03  2.172440e-02  2.915830e-02  1.773087e-02   
3               1.359208e-01  1.441813e-01  9.518037e-02  5.642293e-02   
4               6.372562e-02  1.618925e-01  2.810865e-02  8.575606e-02   
...                      ...           ...           ...           ...   
7421            1.606871e-10  3.973695e-12  9.368230e-09  1.448191e-11   
7422            1.111438e-09  3.635185e-11  2.420178e-08  3.789151e-11   
7423            2.290258e-10  7.493064e-03  7.493064e-03  2.418648e-03   
7424            9.283623e-11  1.113672e-02  1.113672e-02  2.348951e-03   
7425            6.026280e-11  3.753684e-12  6.330155e-08  1.060724e-10   

                             dataset tile_id field_id  
field_neigbour         